In [111]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorIndexer
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.sql.functions import split
from pyspark.sql.functions import col, hour, minute, second, count,isnan, when, month

In [112]:
spark = SparkSession.builder.appName('preprocessing').getOrCreate()

In [113]:
tweet_df = spark.read.option("header", "true").option("delimiter", "\t") \
    .option("inferSchema", "true").csv("data/tweet.txt")

In [114]:
tweet_df.printSchema()

root
 |-- tweet_id: integer (nullable = true)
 |-- tweet_tweetId: long (nullable = true)
 |-- tweet_userId: integer (nullable = true)
 |-- tweet_artistId: integer (nullable = true)
 |-- tweet_trackId: integer (nullable = true)
 |-- tweet_datetime: timestamp (nullable = true)
 |-- tweet_weekday: integer (nullable = true)
 |-- tweet_longitude: double (nullable = true)
 |-- tweet_latitude: double (nullable = true)



In [118]:
tweet_new = tweet_df.withColumn("total_seconds", hour("tweet_datetime") * 3600 + minute("tweet_datetime") * 60 + second("tweet_datetime"))
tweet_new = tweet_new.withColumn("month", month("tweet_datetime"))

In [119]:
tweet_new.printSchema()
# możemy dropnąć twet_tweetId
# i to tweet_datetime ??? (bo już raczej niepotrzebne)

root
 |-- tweet_id: integer (nullable = true)
 |-- tweet_tweetId: long (nullable = true)
 |-- tweet_userId: integer (nullable = true)
 |-- tweet_artistId: integer (nullable = true)
 |-- tweet_trackId: integer (nullable = true)
 |-- tweet_datetime: timestamp (nullable = true)
 |-- tweet_weekday: integer (nullable = true)
 |-- tweet_longitude: double (nullable = true)
 |-- tweet_latitude: double (nullable = true)
 |-- total_seconds: integer (nullable = true)
 |-- month: integer (nullable = true)



In [120]:
tweet_new.show(10)

+--------+------------------+------------+--------------+-------------+-------------------+-------------+---------------+--------------+-------------+-----+
|tweet_id|     tweet_tweetId|tweet_userId|tweet_artistId|tweet_trackId|     tweet_datetime|tweet_weekday|tweet_longitude|tweet_latitude|total_seconds|month|
+--------+------------------+------------+--------------+-------------+-------------------+-------------+---------------+--------------+-------------+-----+
|       1|167408118735699971|    24842995|         25040|       420585|2012-02-09 00:43:00|            3|       0.126573|       52.1976|         2580|    2|
|       2|167408251737088000|    77147006|        122992|      2106213|2012-02-09 00:43:32|            3|       -80.1999|       36.0349|         2612|    2|
|       3|185189790390558720|   174626103|        468091|      8030020|2012-03-29 02:21:01|            3|      -0.142822|       51.5207|         8461|    3|
|       4|229940340692500480|   538669914|         67904| 

### check if missing

In [121]:
c_tab = ["tweet_id","tweet_userId","tweet_artistId","tweet_trackId","tweet_weekday","tweet_longitude","tweet_latitude","total_seconds"]
missing= tweet_new.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in c_tab])

In [122]:
missing.show()

+--------+------------+--------------+-------------+-------------+---------------+--------------+-------------+
|tweet_id|tweet_userId|tweet_artistId|tweet_trackId|tweet_weekday|tweet_longitude|tweet_latitude|total_seconds|
+--------+------------+--------------+-------------+-------------+---------------+--------------+-------------+
|       0|           0|             0|            0|            0|              0|             0|            0|
+--------+------------+--------------+-------------+-------------+---------------+--------------+-------------+



In [123]:
# możemy dropnąć 
# możemy dropnąć twet_tweetId
# i to tweet_datetime 
col_drop = ["tweet_tweetId","tweet_datetime"]
tweet_new = tweet_new.drop(*col_drop)

In [124]:
tweet_new.printSchema()

root
 |-- tweet_id: integer (nullable = true)
 |-- tweet_userId: integer (nullable = true)
 |-- tweet_artistId: integer (nullable = true)
 |-- tweet_trackId: integer (nullable = true)
 |-- tweet_weekday: integer (nullable = true)
 |-- tweet_longitude: double (nullable = true)
 |-- tweet_latitude: double (nullable = true)
 |-- total_seconds: integer (nullable = true)
 |-- month: integer (nullable = true)



## Encoding

In [125]:
# zmiana numeracji dni tyg, 1-pon, 2-wt itd
tweet_new = tweet_new.withColumn("tweet_weekday", col("tweet_weekday") + 1)

In [126]:
categor_cols = ["tweet_weekday","month"]
all_stages = []

for categor_col in categor_cols:
    encoder = OneHotEncoder(inputCols=[categor_col], outputCols=[categor_col + "_cat_vec"])
    all_stages += [encoder]

numeric_cols = ["tweet_longitude","tweet_latitude","total_seconds"]
assembler_inputs = [c + "_cat_vec" for c in categor_cols] + numeric_cols
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
all_stages += [assembler]

In [127]:
all_stages

[OneHotEncoder_3f0e036c71ac,
 OneHotEncoder_29c7e3fa3241,
 VectorAssembler_e86e16698f0f]

In [128]:
tweet_new2 = Pipeline(stages=all_stages).fit(tweet_new).transform(tweet_new)

In [129]:
tweet_new2.printSchema()

root
 |-- tweet_id: integer (nullable = true)
 |-- tweet_userId: integer (nullable = true)
 |-- tweet_artistId: integer (nullable = true)
 |-- tweet_trackId: integer (nullable = true)
 |-- tweet_weekday: integer (nullable = true)
 |-- tweet_longitude: double (nullable = true)
 |-- tweet_latitude: double (nullable = true)
 |-- total_seconds: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- tweet_weekday_cat_vec: vector (nullable = true)
 |-- month_cat_vec: vector (nullable = true)
 |-- features: vector (nullable = true)



In [130]:
tweet_new2.select("features").where(col("tweet_id") == 1).first()[0]

SparseVector(22, {4: 1.0, 9: 1.0, 19: 0.1266, 20: 52.1976, 21: 2580.0})